## imports, data loading, & data prep.

In [1]:
import Pkg
include("EVGONN.jl")
include("NN.jl")
using .EVGONN
using .NN
using StatsBase
using MLDatasets

In [50]:
n_categories = 10;
n_var = 784;
n = 60000;

In [51]:
train_x, train_y = FashionMNIST.traindata()
test_x, test_y  = FashionMNIST.testdata();

In [52]:
X_train = reshape(convert(Array{Float64}, train_x), (n_var, n));
X_test = reshape(convert(Array{Float64}, test_x), (n_var, 10000));
X = hcat(X_train, X_test);

In [53]:
dt = fit(ZScoreTransform, Array(X), dims=1)
X = StatsBase.transform(dt, X);

In [54]:
X_train = reshape(X[:,1:60000], (n_var, 1, 60000));
X_test = reshape(X[:,60001:70000], (n_var, 1, 10000));

In [76]:
train_y = train_y.+ 1;
y_test = test_y.+ 1;

In [78]:
y_train = zeros(UInt8, (n, n_categories))
for i in 1:n_train
    y_train[i, train_y[i]] = 0x01
end

## model exploration.

In [3]:
# optimize each separately and see overall which can attain better accuracy
# explore # of hidden layers, # nodes, and experiment with changing learning rate over time
# also compare # of iterations and running time

In [127]:
nn_van = NN.NeuralNetwork(n_var, (40,20,20), n_categories, η=0.01);
nn_evgo = EVGONN.NeuralNetwork(n_var, (40,20,20), n_categories, 3, η=0.01, β1=0.01, β2=0.0000001);

In [95]:
n = 60000

60000

In [39]:
function training(iters=10000)
    costs = zeros(n);
    lr_up(x) = min(0.00000000005x^2 + 0.001, 0.51)
    nn_van.learning_rate = 0.005
    for it in 1:iters
        for i in 1:n
            result = NN.train(NN.prepare(X_train[:, 1, i]'), NN.prepare(y_train[i, :]'), nn_van)
            costs[i] = result["cost"]
        end
        nn_van.learning_rate = lr_up(it)
        if it % 10 == 0
            println("error = ", sum(costs))
        end
    end
end

training (generic function with 2 methods)

In [40]:
training()

error = 196.1046773957724
error = 195.9388902765378
error = 195.8657693201574
error = 195.80151496103454
error = 195.7417236667833
error = 195.684628314463


LoadError: InterruptException:

In [83]:
function check()
    checks = Dict("HIT" => 0, "MISS" => 0)
    for i in 1:10000
        if argmax(NN.predict(NN.prepare(X_test[:, 1, i]'), nn_van)["result"])[2] == y_test[i]
            checks["HIT"] += 1
        else
            checks["MISS"] += 1
        end
    end
    println("accuracy = $(checks["HIT"] / 10000)%")
end

check (generic function with 1 method)

In [84]:
check()

accuracy = 0.8181%
Dict("MISS" => 1819,"HIT" => 8181)


In [90]:
function check(nn)
    checks = Dict("HIT" => 0, "MISS" => 0)
    for i in 1:10000
        if argmax(EVGONN.predict(EVGONN.prepare(X_test[:, 1, i]'), nn)["result"])[2] == y_test[i]
            checks["HIT"] += 1
        else
            checks["MISS"] += 1
        end
    end
    return checks["HIT"] / 10000
end

check (generic function with 1 method)

In [129]:
function training(iters=10000)
    costs = zeros(n)
    old_costs = sum(costs)
    #lr_up(x) = min(0.0000000005x^2 + 0.01, 0.51)
    for it in 1:iters
        for i in 1:n
            result = EVGONN.train(EVGONN.prepare(X_train[:, 1, i]'), EVGONN.prepare(y_train[i, :]'), nn_evgo)
            costs[i] = result["cost"]
        end
        #nn_evgo.learning_rate = lr_up(it)
        if it % 10 == 0
            println("error = ", sum(costs))
        end
    end
end

training (generic function with 2 methods)

In [ ]:
training()

error = 14109.02550399244
error = 12395.39323745981
error = 11638.482452913066
error = 10887.420602344482
error = 10372.743525521475
error = 9842.5310931854


basically training is fucking slow for both (had to train on only 10k to even make it reasonable) but evgo does it in way less epochs. for evgo loss decreases more consistently but each epoch is longer. which makes sense. it's also less sensitive to initialization obviously. and i didn't have to babysit the learning rate. honestly the convergence was pretty sexy just have to be patient, regular gd was either really slow or too bouncy in the 190s but evgo handled it really well. i think it's overfitting though, probably need to run on full 60k